In [1]:
from  pyspark.sql.functions import input_file_name
import os

df = spark.read.load('abfss://hddata@[container].dfs.core.windows.net/[path]/*.csv', format='csv', header=True).withColumn('file', input_file_name())

display(df)

StatementMeta(wu2sinespark, 31, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, b9f699e2-ec15-4c1c-9181-9c59f1d2dd7a)


## Profile code for new dataset


In [2]:
df_profile = df.describe()


StatementMeta(wu2sinespark, 31, 4, Finished, Available)

## Save out any additional validation data sets 


In [3]:
validation_count_by_date = df.groupBy('file','date').count()


StatementMeta(wu2sinespark, 31, 5, Finished, Available)

In [4]:
display(validation_count_by_date)

StatementMeta(wu2sinespark, 31, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0011ff6e-dfba-49e9-9a0d-23043d5796ce)

## Count missing values


In [5]:
from pyspark.sql.functions import isnull, isnan, when, count, col

missing_values = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T
cols = [count(when(isnan(c) | col(c).isNull(), c)).alias(c) if c not in ['file', 'date'] else count(col(c)).alias('count_{}'.format(c)) for c in df.columns]
missing_by_file = df.groupBy('file', 'date').agg(*cols)

display(missing_by_file)


StatementMeta(wu2sinespark, 31, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 07ddc2ea-02c4-43ab-a265-c9497ebcc9da)

## Calculate missing value percentage for each field grouped by file/date


In [9]:
missing_by_file_pct = missing_by_file.select(['date', 'file', *[(col(c)/col('count_file')).alias('{}_pct'.format(c)) for c in df.columns]])

StatementMeta(wu2sinespark, 31, 11, Finished, Available)

In [10]:
display(missing_by_file_pct)

StatementMeta(wu2sinespark, 31, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, cd69a3bf-0063-40a2-9847-fb3d43076282)

## Count distinct models by date


In [8]:
from pyspark.sql.functions import countDistinct

validation_modelcount_by_date = df.groupBy('file','date').agg(countDistinct('model'))

display(validation_modelcount_by_date)

StatementMeta(wu2sinespark, 31, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 39fc234d-0984-4590-8ec5-bb807b98cfb2)

## 
